In [ ]:
import os
import platform
import mysql.connector
import datetime
import random
import sys
import decimal

try:
    mydb = mysql.connector.connect(
        user='root',
        password='abcd1234',
        host='localhost'
    )
    if mydb.is_connected():
        print("Connection Successful")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    sys.exit(1)

mycursor = mydb.cursor()

def create_database():
    try:
        mycursor.execute("DROP DATABASE IF EXISTS blood_bank")
        mycursor.execute("CREATE DATABASE blood_bank")
        mycursor.execute("USE blood_bank")
        print("Database 'blood_bank' created successfully")
    except mysql.connector.Error as err:
        print(f"Error creating database: {err}")
        sys.exit(1)

def create_tables():
    try:
        mycursor.execute("DROP TABLE IF EXISTS blood_request")
        mycursor.execute("DROP TABLE IF EXISTS donation")
        mycursor.execute("DROP TABLE IF EXISTS blood_stock")
        mycursor.execute("DROP TABLE IF EXISTS blood_camp")
        mycursor.execute("DROP TABLE IF EXISTS recipient")
        mycursor.execute("DROP TABLE IF EXISTS donor")
        mycursor.execute("DROP TABLE IF EXISTS organisation")
        
        mycursor.execute("""
        CREATE TABLE organisation(
            oid INT PRIMARY KEY AUTO_INCREMENT,
            name VARCHAR(50) NOT NULL,
            contact_no VARCHAR(15) NOT NULL,
            email VARCHAR(50) UNIQUE,
            license_no VARCHAR(20) UNIQUE,
            state VARCHAR(20),
            address VARCHAR(100),
            city VARCHAR(20),
            established_date DATE,
            CONSTRAINT chk_email CHECK (email LIKE '%@%.%')
        )""")
        
        mycursor.execute("""
        CREATE TABLE donor(
            donor_id INT PRIMARY KEY AUTO_INCREMENT,
            name VARCHAR(50) NOT NULL,
            phone_no VARCHAR(15) NOT NULL UNIQUE,
            email VARCHAR(50) UNIQUE,
            dob DATE NOT NULL,
            password VARCHAR(100) NOT NULL,
            gender ENUM('Male','Female','Other') NOT NULL,
            blood_group ENUM('A+','A-','B+','B-','AB+','AB-','O+','O-') NOT NULL,
            weight DECIMAL(5,2) CHECK (weight >= 50),
            last_donation_date DATE,
            is_active BOOLEAN DEFAULT TRUE,
            registration_date DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        mycursor.execute("""
        CREATE TABLE recipient(
            recipient_id INT PRIMARY KEY AUTO_INCREMENT,
            name VARCHAR(50) NOT NULL,
            phone_no VARCHAR(15) NOT NULL UNIQUE,
            email VARCHAR(50) UNIQUE,
            hospital_name VARCHAR(50),
            doctor_name VARCHAR(50),
            registration_date DATETIME DEFAULT CURRENT_TIMESTAMP
        )""")
        
        mycursor.execute("""
        CREATE TABLE blood_camp(
            camp_id INT PRIMARY KEY AUTO_INCREMENT,
            oid INT NOT NULL,
            camp_name VARCHAR(50) NOT NULL,
            start_datetime DATETIME NOT NULL,
            end_datetime DATETIME NOT NULL,
            address VARCHAR(100) NOT NULL,
            expected_donors INT,
            actual_donors INT DEFAULT 0,
            status ENUM('Planned','Ongoing','Completed','Cancelled'),
            FOREIGN KEY (oid) REFERENCES organisation(oid),
            CONSTRAINT chk_dates CHECK (end_datetime > start_datetime)
        )""")
        
        mycursor.execute("""
        CREATE TABLE blood_stock(
            stock_id INT PRIMARY KEY AUTO_INCREMENT,
            oid INT NOT NULL,
            blood_group ENUM('A+','A-','B+','B-','AB+','AB-','O+','O-') NOT NULL,
            quantity_ml INT NOT NULL CHECK (quantity_ml >= 0),
            collection_date DATE NOT NULL,
            expiry_date DATE NOT NULL,
            storage_location VARCHAR(20),
            status ENUM('Available','Reserved','Expired','Used'),
            FOREIGN KEY (oid) REFERENCES organisation(oid),
            CONSTRAINT chk_expiry CHECK (expiry_date > collection_date)
        )""")
        
        mycursor.execute("""
        CREATE TABLE donation(
            donation_id INT PRIMARY KEY AUTO_INCREMENT,
            donor_id INT NOT NULL,
            camp_id INT,
            oid INT NOT NULL,
            donation_date DATETIME DEFAULT CURRENT_TIMESTAMP,
            blood_group ENUM('A+','A-','B+','B-','AB+','AB-','O+','O-') NOT NULL,
            quantity_ml INT NOT NULL CHECK (quantity_ml BETWEEN 350 AND 450),
            hemoglobin_level DECIMAL(3,1) NOT NULL,
            blood_pressure VARCHAR(10) NOT NULL,
            notes TEXT,
            FOREIGN KEY (donor_id) REFERENCES donor(donor_id),
            FOREIGN KEY (camp_id) REFERENCES blood_camp(camp_id),
            FOREIGN KEY (oid) REFERENCES organisation(oid)
        )""")
        
        mycursor.execute("""
        CREATE TABLE blood_request(
            request_id INT PRIMARY KEY AUTO_INCREMENT,
            recipient_id INT NOT NULL,
            oid INT NOT NULL,
            blood_group ENUM('A+','A-','B+','B-','AB+','AB-','O+','O-') NOT NULL,
            quantity_ml INT NOT NULL CHECK (quantity_ml > 0),
            request_date DATETIME DEFAULT CURRENT_TIMESTAMP,
            required_date DATE NOT NULL,
            priority ENUM('Emergency','High','Normal'),
            status ENUM('Pending','Approved','Rejected','Fulfilled'),
            doctor_approval VARCHAR(50),
            notes TEXT,
            FOREIGN KEY (recipient_id) REFERENCES recipient(recipient_id),
            FOREIGN KEY (oid) REFERENCES organisation(oid)
        )""")
        
        mycursor.execute("""
        CREATE TABLE system_users (
            user_id INT PRIMARY KEY AUTO_INCREMENT,
            username VARCHAR(30) UNIQUE NOT NULL,
            password VARCHAR(100) NOT NULL,
            full_name VARCHAR(50) NOT NULL,
            role ENUM('Admin','Staff','Volunteer') NOT NULL,
            email VARCHAR(50) UNIQUE,
            last_login DATETIME,
            is_active BOOLEAN DEFAULT TRUE,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        print("8 tables created successfully")
    except mysql.connector.Error as err:
        print(f"Error creating tables: {err}")

def insert_sample_data():
    try:
        organisations = [
            ("LifeBlood Foundation", "9876543210", "info@lifeblood.org", "LB123456", "California", "123 Health St, Suite 100", "Los Angeles", "2005-06-15"),
            ("Red Cross Society", "8765432109", "contact@redcross.org", "RC654321", "New York", "456 Aid Avenue", "New York", "1980-03-20"),
            ("BloodCare Network", "7654321098", "support@bloodcare.net", "BC789012", "Texas", "789 Donor Plaza", "Houston", "2010-11-05"),
            ("Hemoglobin Alliance", "6543210987", "help@hemoalliance.com", "HA345678", "Florida", "321 Plasma Road", "Miami", "1995-08-12"),
            ("VitalFlow Centers", "5432109876", "admin@vitalflow.org", "VF901234", "Illinois", "654 Transfusion Tower", "Chicago", "2008-04-30"),
            ("BloodLink Systems", "4321098765", "info@bloodlink.sys", "BL567890", "Pennsylvania", "987 Vein Valley", "Philadelphia", "2015-07-22"),
            ("The Blood Bank", "3210987654", "contact@thebloodbank.org", "TB234567", "Ohio", "159 Circulation Circle", "Columbus", "2000-01-18"),
            ("Plasma Partners", "2109876543", "support@plasmapartners.com", "PP890123", "Georgia", "753 Bloodstream Blvd", "Atlanta", "2012-09-14"),
            ("Type O Network", "1098765432", "info@typeo.net", "ON456789", "Michigan", "486 Group O Drive", "Detroit", "2003-05-09"),
            ("Universal Donors", "0987654321", "contact@universaldonors.org", "UD012345", "North Carolina", "852 Universal Way", "Charlotte", "1998-12-25"),
            ("Blood Warriors", "9876543211", "help@bloodwarriors.com", "BW678901", "Washington", "369 Battle Lane", "Seattle", "2007-02-28"),
            ("SaveLife Foundation", "8765432101", "info@savelife.org", "SL234567", "Massachusetts", "741 Rescue Road", "Boston", "1990-10-11"),
            ("Blood for All", "7654321091", "support@bloodforall.net", "BA890123", "Arizona", "963 Donation Drive", "Phoenix", "2006-08-17"),
            ("The Giving Vein", "6543210981", "contact@givingvein.org", "GV456789", "Colorado", "258 Altruism Avenue", "Denver", "2018-04-03"),
            ("Hemoglobin Heroes", "5432109871", "info@hemoheroes.com", "HH012345", "Oregon", "147 Lifesaver Lane", "Portland", "2009-07-19")
        ]
        mycursor.executemany("""
        INSERT INTO organisation (name, contact_no, email, license_no, state, address, city, established_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, organisations)
        
        donors = [
            ("John Smith", "1234567890", "john.smith@email.com", "1990-05-15", "Male", "password123", "A+", 72.5, "2023-01-10", True),
            ("Emily Johnson", "2345678901", "emily.j@email.com", "1985-08-22", "Female", "securepass", "B-", 68.2, "2023-02-15", True),
            ("Michael Brown", "3456789012", "michael.b@email.com", "1995-03-30", "Male", "myp@ssword", "O+", 80.0, None, True),
            ("Sarah Davis", "4567890123", "sarah.d@email.com", "1988-11-05", "Female", "sarahpass", "AB+", 65.7, "2023-03-20", True),
            ("Robert Wilson", "5678901234", "robert.w@email.com", "1992-07-18", "Male", "robert123", "A-", 75.3, "2022-12-05", True),
            ("Jennifer Lee", "6789012345", "jennifer.l@email.com", "1987-04-25", "Female", "jennypass", "B+", 63.8, "2023-04-01", True),
            ("David Miller", "7890123456", "david.m@email.com", "1993-09-12", "Male", "davidpass", "O-", 82.1, None, True),
            ("Jessica Taylor", "8901234567", "jessica.t@email.com", "1989-06-28", "Female", "jesspass", "AB-", 67.5, "2023-01-30", True),
            ("Christopher Anderson", "9012345678", "chris.a@email.com", "1991-02-14", "Male", "chris123", "A+", 78.9, "2022-11-15", True),
            ("Amanda Martinez", "0123456789", "amanda.m@email.com", "1994-10-08", "Female", "amandapass", "B-", 70.2, None, True),
            ("Daniel Thomas", "1122334455", "daniel.t@email.com", "1986-12-03", "Male", "danielpass", "O+", 85.0, "2023-03-10", True),
            ("Elizabeth White", "2233445566", "elizabeth.w@email.com", "1996-01-20", "Female", "elizpass", "AB+", 64.3, None, True),
            ("Matthew Harris", "3344556677", "matthew.h@email.com", "1984-07-07", "Male", "mattpass", "A-", 76.8, "2023-02-28", True),
            ("Ashley Clark", "4455667788", "ashley.c@email.com", "1997-05-19", "Female", "ashleypass", "B+", 69.5, None, True),
            ("James Lewis", "5566778899", "james.l@email.com", "1983-08-31", "Male", "jamespass", "O-", 83.2, "2023-04-05", True)
        ]
        mycursor.executemany("""
        INSERT INTO donor (name, phone_no, email, dob, gender, password, blood_group, weight, last_donation_date, is_active)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, donors)
        
        recipients = [
            ("Michael Anderson", "9988776655", "michael.anderson@email.com", "City General Hospital", "Dr. Smith"),
            ("Sarah Wilson", "8877665544", "sarah.wilson@email.com", "Metro Health Center", "Dr. Johnson"),
            ("David Thompson", "7766554433", "david.thompson@email.com", "Sunshine Hospital", "Dr. Williams"),
            ("Jennifer Martinez", "6655443322", "jennifer.martinez@email.com", "Hope Medical Center", "Dr. Brown"),
            ("Robert Taylor", "5544332211", "robert.taylor@email.com", "LifeCare Hospital", "Dr. Davis"),
            ("Emily Garcia", "4433221100", "emily.garcia@email.com", "Community Health", "Dr. Miller"),
            ("Christopher Lee", "3322110099", "christopher.lee@email.com", "Regional Medical", "Dr. Wilson"),
            ("Jessica Clark", "2211009988", "jessica.clark@email.com", "University Hospital", "Dr. Moore"),
            ("Daniel Rodriguez", "1100998877", "daniel.rodriguez@email.com", "Children's Hospital", "Dr. Taylor"),
            ("Amanda Lewis", "0099887766", "amanda.lewis@email.com", "Mercy Hospital", "Dr. Anderson"),
            ("Matthew Walker", "1199228833", "matthew.walker@email.com", "St. Mary's Hospital", "Dr. Thomas"),
            ("Ashley Hall", "2288337744", "ashley.hall@email.com", "Memorial Hospital", "Dr. Jackson"),
            ("James Allen", "3377446655", "james.allen@email.com", "Parkview Medical", "Dr. White"),
            ("Elizabeth Young", "4466558877", "elizabeth.young@email.com", "Valley Hospital", "Dr. Harris"),
            ("Joshua Hernandez", "5555666677", "joshua.hernandez@email.com", "Riverside Medical", "Dr. Martin")
        ]
        
        mycursor.executemany("""
        INSERT INTO recipient (name, phone_no, email, hospital_name, doctor_name)
        VALUES (%s, %s, %s, %s, %s)
        """, recipients)
        
        blood_camps = [
            (1, "Spring Blood Drive 2023", "2023-04-15 09:00:00", "2023-04-15 17:00:00", "123 Health St, Los Angeles", 100, 85, "Completed"),
            (2, "Summer Donation Camp", "2023-06-20 10:00:00", "2023-06-20 18:00:00", "456 Aid Avenue, New York", 120, 110, "Completed"),
            (3, "Fall Blood Donation", "2023-09-10 08:00:00", "2023-09-10 16:00:00", "789 Donor Plaza, Houston", 80, 75, "Completed"),
            (4, "Winter Blood Drive", "2023-12-05 09:30:00", "2023-12-05 17:30:00", "321 Plasma Road, Miami", 90, 78, "Completed"),
            (5, "Community Donation Day", "2023-05-22 08:00:00", "2023-05-22 16:00:00", "654 Transfusion Tower, Chicago", 70, 65, "Completed"),
            (6, "Corporate Blood Drive", "2023-07-18 10:00:00", "2023-07-18 18:00:00", "987 Vein Valley, Philadelphia", 60, 55, "Completed"),
            (7, "University Blood Camp", "2023-08-25 09:00:00", "2023-08-25 17:00:00", "159 Circulation Circle, Columbus", 150, 140, "Completed"),
            (8, "Emergency Blood Drive", "2023-03-12 08:30:00", "2023-03-12 16:30:00", "753 Bloodstream Blvd, Atlanta", 50, 48, "Completed"),
            (9, "Annual Donation Camp", "2023-11-15 09:00:00", "2023-11-15 17:00:00", "486 Group O Drive, Detroit", 200, 180, "Completed"),
            (10, "Holiday Blood Drive", "2023-12-20 10:00:00", "2023-12-20 18:00:00", "852 Universal Way, Charlotte", 75, 70, "Completed"),
            (11, "Spring Donation Event", "2023-04-08 08:00:00", "2023-04-08 16:00:00", "369 Battle Lane, Seattle", 85, 80, "Completed"),
            (12, "Summer Blood Camp", "2023-07-05 09:00:00", "2023-07-05 17:00:00", "741 Rescue Road, Boston", 95, 90, "Completed"),
            (13, "Fall Donation Drive", "2023-10-12 08:30:00", "2023-10-12 16:30:00", "963 Donation Drive, Phoenix", 65, 60, "Completed"),
            (14, "Winter Donation Day", "2023-01-20 09:00:00", "2023-01-20 17:00:00", "258 Altruism Avenue, Denver", 55, 50, "Completed"),
            (15, "New Year Blood Drive", "2023-01-05 10:00:00", "2023-01-05 18:00:00", "147 Lifesaver Lane, Portland", 40, 38, "Completed")
        ]
        mycursor.executemany("""
        INSERT INTO blood_camp (oid, camp_name, start_datetime, end_datetime, address, expected_donors, actual_donors, status)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, blood_camps)
        
        blood_stock = [
            (1, "A+", 500, "2023-04-15", "2023-05-27", "Shelf A1", "Available"),
            (2, "B-", 450, "2023-06-20", "2023-08-01", "Shelf B2", "Available"),
            (3, "O+", 600, "2023-09-10", "2023-10-22", "Shelf C3", "Available"),
            (4, "AB+", 350, "2023-12-05", "2024-01-16", "Shelf D4", "Available"),
            (5, "A-", 400, "2023-05-22", "2023-07-03", "Shelf E5", "Available"),
            (6, "B+", 550, "2023-07-18", "2023-08-29", "Shelf F6", "Available"),
            (7, "O-", 300, "2023-08-25", "2023-10-06", "Shelf G7", "Available"),
            (8, "AB-", 250, "2023-03-12", "2023-04-23", "Shelf H8", "Expired"),
            (9, "A+", 700, "2023-11-15", "2023-12-27", "Shelf I9", "Available"),
            (10, "B-", 380, "2023-12-20", "2024-01-31", "Shelf J10", "Available"),
            (11, "O+", 650, "2023-04-08", "2023-05-20", "Shelf K11", "Expired"),
            (12, "AB+", 420, "2023-07-05", "2023-08-16", "Shelf L12", "Available"),
            (13, "A-", 480, "2023-10-12", "2023-11-23", "Shelf M13", "Available"),
            (14, "B+", 520, "2023-01-20", "2023-03-03", "Shelf N14", "Expired"),
            (15, "O-", 370, "2023-01-05", "2023-02-16", "Shelf O15", "Expired")
        ]
        mycursor.executemany("""
        INSERT INTO blood_stock (oid, blood_group, quantity_ml, collection_date, expiry_date, storage_location, status)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, blood_stock)
        
        donations = [
            (1, 1, 1, "2023-04-15 10:30:00", "A+", 450, 14.5, "120/80", "Regular donor"),
            (2, 2, 2, "2023-06-20 11:15:00", "B-", 400, 13.8, "118/76", "First-time donor"),
            (3, 3, 3, "2023-09-10 09:45:00", "O+", 450, 15.2, "122/78", ""),
            (4, 4, 4, "2023-12-05 14:20:00", "AB+", 350, 14.0, "119/77", ""),
            (5, 5, 5, "2023-05-22 10:00:00", "A-", 420, 13.5, "121/79", ""),
            (6, 6, 6, "2023-07-18 13:30:00", "B+", 380, 14.8, "117/75", ""),
            (7, 7, 7, "2023-08-25 11:45:00", "O-", 400, 15.0, "123/81", ""),
            (8, 8, 8, "2023-03-12 09:15:00", "AB-", 360, 13.7, "120/80", ""),
            (9, 9, 9, "2023-11-15 15:00:00", "A+", 450, 14.2, "119/78", ""),
            (10, 10, 10, "2023-12-20 12:30:00", "B-", 390, 13.9, "118/76", ""),
            (11, 11, 11, "2023-04-08 10:45:00", "O+", 430, 15.1, "122/80", ""),
            (12, 12, 12, "2023-07-05 14:15:00", "AB+", 370, 14.3, "120/78", ""),
            (13, 13, 13, "2023-10-12 09:30:00", "A-", 410, 13.6, "119/77", ""),
            (14, 14, 14, "2023-01-20 11:00:00", "B+", 440, 14.7, "121/79", ""),
            (15, 15, 15, "2023-01-05 13:45:00", "O-", 380, 15.3, "124/82", "")
        ]
        mycursor.executemany("""
        INSERT INTO donation (donor_id, camp_id, oid, donation_date, blood_group, quantity_ml, hemoglobin_level, blood_pressure, notes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, donations)
        
        blood_requests = [
            (1, 1, "A+", 300, "2023-04-16 08:30:00", "2023-04-17", "Emergency", "Fulfilled", "Dr. Smith", "Emergency surgery"),
            (2, 2, "B-", 250, "2023-06-21 10:15:00", "2023-06-23", "High", "Fulfilled", "Dr. Johnson", ""),
            (3, 3, "O+", 400, "2023-09-11 09:00:00", "2023-09-13", "Normal", "Fulfilled", "Dr. Williams", ""),
            (4, 4, "AB+", 300, "2023-12-06 14:45:00", "2023-12-08", "High", "Fulfilled", "Dr. Brown", ""),
            (5, 5, "A-", 350, "2023-05-23 11:30:00", "2023-05-25", "Normal", "Fulfilled", "Dr. Davis", ""),
            (6, 6, "B+", 200, "2023-07-19 13:15:00", "2023-07-21", "Emergency", "Fulfilled", "Dr. Miller", "Accident victim"),
            (7, 7, "O-", 250, "2023-08-26 10:00:00", "2023-08-28", "High", "Fulfilled", "Dr. Wilson", ""),
            (8, 8, "AB-", 200, "2023-03-13 08:45:00", "2023-03-15", "Normal", "Fulfilled", "Dr. Moore", ""),
            (9, 9, "A+", 500, "2023-11-16 15:30:00", "2023-11-18", "Emergency", "Fulfilled", "Dr. Taylor", "Major surgery"),
            (10, 10, "B-", 300, "2023-12-21 12:15:00", "2023-12-23", "High", "Fulfilled", "Dr. Anderson", ""),
            (11, 11, "O+", 450, "2023-04-09 11:00:00", "2023-04-11", "Normal", "Fulfilled", "Dr. Thomas", ""),
            (12, 12, "AB+", 350, "2023-07-06 14:30:00", "2023-07-08", "High", "Fulfilled", "Dr. Jackson", ""),
            (13, 13, "A-", 400, "2023-10-13 09:15:00", "2023-10-15", "Normal", "Fulfilled", "Dr. White", ""),
            (14, 14, "B+", 300, "2023-01-21 10:45:00", "2023-01-23", "Emergency", "Fulfilled", "Dr. Harris", "Critical condition"),
            (15, 15, "O-", 250, "2023-01-06 13:30:00", "2023-01-08", "High", "Fulfilled", "Dr. Martin", "")
        ]
        mycursor.executemany("""
        INSERT INTO blood_request (recipient_id, oid, blood_group, quantity_ml, request_date, required_date, priority, status, doctor_approval, notes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, blood_requests)
        
        
        organisations = [
            ("LifeBlood Foundation", "9876543210", "contact@lifebloodfoundation.org", "LB1234561", "California", "123 Health St, Suite 100", "Los Angeles", "2005-06-15"),
            ("Red Cross Society", "8765432109", "donations@redcrosssociety.org", "RC6543211", "New York", "456 Aid Avenue", "New York", "1980-03-20"),
            ("BloodCare Network", "7654321098", "support@bloodcarenetwork.org", "BC7890121", "Texas", "789 Donor Plaza", "Houston", "2010-11-05"),
            ("Hemoglobin Alliance", "6543210987", "info@hemoglobinal.org", "HA3456781", "Florida", "321 Plasma Road", "Miami", "1995-08-12"),
            ("VitalFlow Centers", "5432109876", "admin@vitalflowcenters.org", "VF9012341", "Illinois", "654 Transfusion Tower", "Chicago", "2008-04-30"),
            ("BloodLink Systems", "4321098765", "contact@bloodlinksystems.com", "BL5678901", "Pennsylvania", "987 Vein Valley", "Philadelphia", "2015-07-22"),
            ("The Blood Bank", "3210987654", "help@thebloodbank.org", "TB2345671", "Ohio", "159 Circulation Circle", "Columbus", "2000-01-18"),
            ("Plasma Partners", "2109876543", "support@plasmapartners.org", "PP8901231", "Georgia", "753 Bloodstream Blvd", "Atlanta", "2012-09-14"),
            ("Type O Network", "1098765432", "donors@typeonetwork.org", "ON4567891", "Michigan", "486 Group O Drive", "Detroit", "2003-05-09"),
            ("Universal Donors", "0987654321", "info@universaldonors.net", "UD0123451", "North Carolina", "852 Universal Way", "Charlotte", "1998-12-25"),
            ("Blood Warriors", "9876543211", "team@bloodwarriors.org", "BW6789011", "Washington", "369 Battle Lane", "Seattle", "2007-02-28"),
            ("SaveLife Foundation", "8765432101", "contact@savelifefdn.org", "SL2345671", "Massachusetts", "741 Rescue Road", "Boston", "1990-10-11"),
            ("Blood for All", "7654321091", "donate@bloodforall.org", "BA8901231", "Arizona", "963 Donation Drive", "Phoenix", "2006-08-17"),
            ("The Giving Vein", "6543210981", "give@thegivingvein.org", "GV4567891", "Colorado", "258 Altruism Avenue", "Denver", "2018-04-03"),
            ("Hemoglobin Heroes", "5432109871", "heroes@hemoglobin.org", "HH0123451", "Oregon", "147 Lifesaver Lane", "Portland", "2009-07-19")
        ]
        mycursor.executemany("""
        INSERT INTO organisation (name, contact_no, email, license_no, state, address, city, established_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, organisations)
        
        mycursor.executemany("""
        INSERT INTO system_users (username, password, full_name, role, email, is_active)
        VALUES (%s, %s, %s, %s, %s, %s)
        """, [
            ("admin", "admin123", "System Administrator", "Admin", "admin@bloodbank.org", True),
            ("staff1", "staff123", "John Doe", "Staff", "staff1@bloodbank.org", True),
            ("vol1", "vol123", "Jane Smith", "Volunteer", "vol1@bloodbank.org", True)
        ])
                
        mydb.commit()
        print("Sample data inserted successfully (15 records in each table)")
    except mysql.connector.Error as err:
        print(f"Error inserting sample data: {err}")

def create_procedures():
    try:
        mycursor.execute("DROP PROCEDURE IF EXISTS check_blood_availability")
        mycursor.execute("""
        CREATE PROCEDURE check_blood_availability(IN bg VARCHAR(5), IN qty INT)
        BEGIN
            DECLARE available_qty INT;
            
            SELECT COALESCE(SUM(quantity_ml), 0) INTO available_qty 
            FROM blood_stock 
            WHERE blood_group = bg AND status = 'Available' AND expiry_date > CURDATE();
            
            IF available_qty >= qty THEN
                SELECT CONCAT('Available: ', available_qty, 'ml') AS result;
            ELSE
                SELECT CONCAT('Insufficient. Available: ', available_qty, 'ml') AS result;
            END IF;
        END
        """)
        
        mycursor.execute("DROP PROCEDURE IF EXISTS get_donor_stats")
        mycursor.execute("""
        CREATE PROCEDURE get_donor_stats(IN org_id INT)
        BEGIN
            SELECT 
                d.blood_group,
                COUNT(*) AS total_donors,
                SUM(CASE WHEN d.last_donation_date IS NULL THEN 1 ELSE 0 END) AS new_donors,
                SUM(CASE WHEN d.last_donation_date IS NOT NULL THEN 1 ELSE 0 END) AS repeat_donors,
                AVG(DATEDIFF(CURDATE(), d.dob)/365) AS avg_age
            FROM donor d
            JOIN donation dn ON d.donor_id = dn.donor_id
            WHERE dn.oid = org_id
            GROUP BY d.blood_group;
        END
        """)
        
        mycursor.execute("DROP PROCEDURE IF EXISTS process_blood_request")
        mycursor.execute("""
        CREATE PROCEDURE process_blood_request(IN req_id INT)
        BEGIN
            DECLARE bg VARCHAR(5);
            DECLARE qty INT;
            DECLARE org_id INT;
            DECLARE available INT;
            
            SELECT blood_group, quantity_ml, oid INTO bg, qty, org_id
            FROM blood_request WHERE request_id = req_id;
            
            CALL check_blood_availability(bg, qty);
            
            SELECT SUM(quantity_ml) INTO available
            FROM blood_stock
            WHERE blood_group = bg AND status = 'Available' AND expiry_date > CURDATE() AND oid = org_id;
            
            IF available >= qty THEN
                UPDATE blood_request SET status = 'Approved' WHERE request_id = req_id;
                
                UPDATE blood_stock 
                SET status = 'Reserved'
                WHERE stock_id IN (
                    SELECT stock_id FROM (
                        SELECT stock_id FROM blood_stock
                        WHERE blood_group = bg AND status = 'Available' AND expiry_date > CURDATE() AND oid = org_id
                        ORDER BY expiry_date
                        LIMIT 1
                    ) AS temp
                );
                
                SELECT 'Request approved' AS result;
            ELSE
                UPDATE blood_request SET status = 'Pending' WHERE request_id = req_id;
                SELECT 'Insufficient stock. Request pending' AS result;
            END IF;
        END
        """)
        
        print("3 procedures created successfully")
    except mysql.connector.Error as err:
        print(f"Error creating procedures: {err}")

def create_triggers():
    try:
        mycursor.execute("DROP TRIGGER IF EXISTS after_donation")
        mycursor.execute("""
        CREATE TRIGGER after_donation
        AFTER INSERT ON donation
        FOR EACH ROW
        BEGIN
            UPDATE donor 
            SET last_donation_date = NEW.donation_date
            WHERE donor_id = NEW.donor_id;
            
            INSERT INTO blood_stock (oid, blood_group, quantity_ml, collection_date, expiry_date, status)
            VALUES (NEW.oid, NEW.blood_group, NEW.quantity_ml, DATE(NEW.donation_date), 
                   DATE_ADD(DATE(NEW.donation_date), INTERVAL 42 DAY), 'Available');
            
            IF NEW.camp_id IS NOT NULL THEN
                UPDATE blood_camp 
                SET actual_donors = actual_donors + 1
                WHERE camp_id = NEW.camp_id;
            END IF;
        END
        """)
        
        mycursor.execute("DROP TRIGGER IF EXISTS before_donation")
        mycursor.execute("""
        CREATE TRIGGER before_donation
        BEFORE INSERT ON donation
        FOR EACH ROW
        BEGIN
            DECLARE last_donation DATE;
            DECLARE donor_age INT;
            
            SELECT last_donation_date, TIMESTAMPDIFF(YEAR, dob, CURDATE())
            INTO last_donation, donor_age
            FROM donor WHERE donor_id = NEW.donor_id;
            
            IF last_donation IS NOT NULL AND DATEDIFF(CURDATE(), last_donation) < 90 THEN
                SIGNAL SQLSTATE '45000'
                SET MESSAGE_TEXT = 'Donor must wait 90 days between donations';
            END IF;
            
            IF donor_age < 18 OR donor_age > 65 THEN
                SIGNAL SQLSTATE '45000'
                SET MESSAGE_TEXT = 'Donor age must be between 18 and 65';
            END IF;
        END
        """)
        
        mycursor.execute("DROP TRIGGER IF EXISTS check_blood_expiry")
        mycursor.execute("""
        CREATE TRIGGER check_blood_expiry
        BEFORE UPDATE ON blood_stock
        FOR EACH ROW
        BEGIN
            IF NEW.expiry_date <= CURDATE() THEN
                SET NEW.status = 'Expired';
            END IF;
        END
        """)
        
        print("3 triggers created successfully")
    except mysql.connector.Error as err:
        print(f"Error creating triggers: {err}")

def main_menu():
    while True:
        print("\nBlood Bank Management System")
        print("1. Donor Management")
        print("2. Recipient Management")
        print("3. Blood Donation")
        print("4. Blood Request")
        print("5. Inventory Management")
        print("6. Reports & Analytics")
        print("7. System Configuration")
        print("8. Exit")
        
        choice = input("Enter choice (1-8): ")
        
        if choice == "1":
            donor_management()
        elif choice == "2":
            recipient_management()
        elif choice == "3":
            blood_donation()
        elif choice == "4":
            blood_request()
        elif choice == "5":
            inventory_management()
        elif choice == "6":
            reports_analytics()
        elif choice == "7":
            system_config()
        elif choice == "8":
            print("Closing system...")
            break
        else:
            print("Invalid choice")

def donor_management():
    while True:
        print("\nDonor Management")
        print("1. Register New Donor")
        print("2. Update Donor Information")
        print("3. View All Donors")
        print("4. Search Donors")
        print("5. View Donation History")
        print("6. Back to Main Menu")
        
        choice = input("Enter choice: ")
        
        if choice == "1":
            register_donor()
        elif choice == "2":
            update_donor()
        elif choice == "3":
            view_all_donors()
        elif choice == "4":
            search_donors()
        elif choice == "5":
            view_donation_history()
        elif choice == "6":
            break
        else:
            print("Invalid choice. Please try again.")

def register_donor():
    print("\nRegister New Donor")
    name = input("Full Name: ")
    phone = input("Phone: ")
    email = input("Email: ")
    dob = input("DOB (YYYY-MM-DD): ")
    gender = input("Gender (Male/Female/Other): ").capitalize()
    blood_group = input("Blood Group (A+, A-, B+, etc.): ").upper()
    weight = float(input("Weight (kg): "))
    
    try:
        mycursor.execute("""
        INSERT INTO donor (name, phone_no, email, dob, gender, password, blood_group, weight)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (name, phone, email, dob, gender, "default123", blood_group, weight))
        donor_id = mycursor.lastrowid
        mydb.commit()
        print(f"Donor registered successfully. Donor ID: {donor_id}")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def update_donor():
    donor_id = input("Enter Donor ID to update: ")
    
    try:
        mycursor.execute("SELECT * FROM donor WHERE donor_id = %s", (donor_id,))
        donor = mycursor.fetchone()
        
        if not donor:
            print("Donor not found")
            return
            
        print("\nCurrent Information:")
        print(f"1. Name: {donor[1]}")
        print(f"2. Phone: {donor[2]}")
        print(f"3. Email: {donor[3]}")
        print(f"4. Blood Group: {donor[7]}")
        print(f"5. Weight: {donor[8]}")
        print(f"6. Active Status: {'Yes' if donor[9] else 'No'}")
        
        field = input("\nEnter field number to update (1-5): ")
        new_value = input("Enter new value: ")
        
        fields = {
            '1': 'name',
            '2': 'phone_no',
            '3': 'email',
            '4': 'blood_group',
            '5': 'weight',
            '6': 'is_active'
        }
        
        if field in fields:
            query = f"UPDATE donor SET {fields[field]} = %s WHERE donor_id = %s"
            mycursor.execute(query, (new_value, donor_id))
            mydb.commit()
            print("Donor information updated successfully")
        else:
            print("Invalid field selection")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def view_all_donors():
    try:
        mycursor.execute("""
        SELECT donor_id, name, phone_no, email, blood_group, 
               last_donation_date, is_active
        FROM donor
        ORDER BY name
        """)
        donors = mycursor.fetchall()
        
        print("\n{:<5} {:<25} {:<15} {:<25} {:<5} {:<15} {:<6}".format(
            "ID", "Name", "Phone", "Email", "BG", "Last Donation", "Active"))
        print("-"*100)
        
        for donor in donors:
            print("{:<5} {:<25} {:<15} {:<25} {:<5} {:<15} {:<6}".format(
                donor[0], donor[1], donor[2], donor[3], donor[4], 
                str(donor[5]) if donor[5] else "Never", "Yes" if donor[6] else "No"))
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def search_donors():
    print("\nSearch Donors By:")
    print("1. Blood Group")
    print("2. Name")
    print("3. Phone Number")
    print("4. Last Donation Date Range")
    
    choice = input("Enter search option: ")
    
    try:
        if choice == "1":
            bg = input("Enter Blood Group: ").upper()
            mycursor.execute("""
            SELECT donor_id, name, phone_no, email, blood_group, last_donation_date
            FROM donor
            WHERE blood_group = %s
            """, (bg,))
        elif choice == "2":
            name = input("Enter Name (partial match): ")
            mycursor.execute("""
            SELECT donor_id, name, phone_no, email, blood_group, last_donation_date
            FROM donor
            WHERE name LIKE %s
            """, (f"%{name}%",))
        elif choice == "3":
            phone = input("Enter Phone Number: ")
            mycursor.execute("""
            SELECT donor_id, name, phone_no, email, blood_group, last_donation_date
            FROM donor
            WHERE phone_no LIKE %s
            """, (f"%{phone}%",))
        elif choice == "4":
            start = input("Enter Start Date (YYYY-MM-DD): ")
            end = input("Enter End Date (YYYY-MM-DD): ")
            mycursor.execute("""
            SELECT donor_id, name, phone_no, email, blood_group, last_donation_date
            FROM donor
            WHERE last_donation_date BETWEEN %s AND %s
            """, (start, end))
        else:
            print("Invalid choice")
            return
            
        results = mycursor.fetchall()
        
        if not results:
            print("No donors found")
            return
            
        print("\nSearch Results:")
        print("{:<5} {:<25} {:<15} {:<25} {:<5} {:<15}".format(
            "ID", "Name", "Phone", "Email", "BG", "Last Donation"))
        print("-"*90)
        
        for row in results:
            print("{:<5} {:<25} {:<15} {:<25} {:<5} {:<15}".format(
                row[0], row[1], row[2], row[3], row[4], 
                str(row[5]) if row[5] else "Never"))
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def view_donation_history():
    donor_id = input("Enter Donor ID: ")
    
    try:
        mycursor.execute("""
        SELECT d.donation_id, d.donation_date, o.name, d.quantity_ml, 
               d.hemoglobin_level, d.blood_pressure
        FROM donation d
        JOIN organisation o ON d.oid = o.oid
        WHERE d.donor_id = %s
        ORDER BY d.donation_date DESC
        """, (donor_id,))
        
        donations = mycursor.fetchall()
        
        if not donations:
            print("No donation history found")
            return
            
        print("\nDonation History:")
        print("{:<5} {:<20} {:<30} {:<8} {:<6} {:<12}".format(
            "ID", "Date", "Organization", "Qty(ml)", "Hb", "BP"))
        print("-"*90)
        
        for donation in donations:
            print("{:<5} {:<20} {:<30} {:<8} {:<6} {:<12}".format(
                donation[0], str(donation[1]), donation[2], 
                donation[3], donation[4], donation[5]))
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def blood_donation():
    print("\nBlood Donation Process")
    donor_id = input("Enter Donor ID: ")
    
    try:
        mycursor.execute("SELECT * FROM donor WHERE donor_id = %s", (donor_id,))
        donor = mycursor.fetchone()
        
        if not donor:
            print("Donor not found")
            return
        
        print(f"\nDonor: {donor[1]}")
        print(f"Blood Group: {donor[7]}")
        print(f"Last Donation: {donor[9] if donor[9] else 'Never'}")
        
        if donor[9]:  
            try:
                if isinstance(donor[9], (datetime.date, datetime.datetime)):
                    last_donation = donor[9].date() if isinstance(donor[9], datetime.datetime) else donor[9]
                elif isinstance(donor[9], str):
                    last_donation = datetime.datetime.strptime(donor[9], '%Y-%m-%d').date()
                elif isinstance(donor[9], decimal.Decimal):
                    last_donation = datetime.date.fromordinal(int(donor[9]))
                else:
                    raise ValueError("Unknown date format in database")
                
                days_since_last = (datetime.date.today() - last_donation).days
                if days_since_last < 90:
                    print(f"\nWarning: Last donation was {days_since_last} days ago (minimum 90 days required)")
                    proceed = input("Do you want to proceed anyway? (y/n): ").lower()
                    if proceed != 'y':
                        return
            except Exception as e:
                print(f"Warning: Could not verify donation frequency - {str(e)}")
                proceed = input("Continue with donation? (y/n): ").lower()
                if proceed != 'y':
                    return
        
        mycursor.execute("SELECT oid, name, city FROM organisation")
        orgs = mycursor.fetchall()
        print("\nAvailable Organizations:")
        for org in orgs:
            print(f"{org[0]}: {org[1]} ({org[2]})")
        
        org_id = input("Select Organization ID: ")
        
        mycursor.execute("""
        SELECT camp_id, camp_name, start_datetime, end_datetime, address 
        FROM blood_camp 
        WHERE oid = %s AND status = 'Ongoing'
        """, (org_id,))
        camps = mycursor.fetchall()
        
        camp_id = None
        if camps:
            print("\nOngoing Camps:")
            for camp in camps:
                print(f"{camp[0]}: {camp[1]} at {camp[4]} ({camp[2]} to {camp[3]})")
            camp_choice = input("Is this donation part of a camp? (y/n): ").lower()
            if camp_choice == 'y':
                camp_id = input("Enter Camp ID: ")
        
        hemoglobin = float(input("Hemoglobin level (g/dL): "))
        if hemoglobin < 12.5:
            print("Warning: Hemoglobin level is below minimum (12.5 g/dL)")
            proceed = input("Do you want to proceed anyway? (y/n): ").lower()
            if proceed != 'y':
                return
        
        bp = input("Blood Pressure (e.g., 120/80): ")
        notes = input("Any notes: ")
        
        mycursor.execute("""
        INSERT INTO donation (donor_id, camp_id, oid, blood_group, quantity_ml, hemoglobin_level, blood_pressure, notes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (donor_id, camp_id, org_id, donor[7], 450, hemoglobin, bp, notes))
        
        mycursor.execute("""
        UPDATE donor 
        SET last_donation_date = CURDATE() 
        WHERE donor_id = %s
        """, (donor_id,))
        
        mydb.commit()
        print("\nDonation recorded successfully")
        
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        mydb.rollback()
    except ValueError as ve:
        print(f"Error: {ve}")
        mydb.rollback()

def recipient_management():
    while True:
        print("\nRecipient Management")
        print("1. Register New Recipient")
        print("2. Update Recipient Information")
        print("3. View All Recipients")
        print("4. Search Recipients")
        print("5. View Request History")
        print("6. Back to Main Menu")
        
        choice = input("Enter choice: ")
        
        if choice == "1":
            register_recipient()
        elif choice == "2":
            update_recipient()
        elif choice == "3":
            view_all_recipients()
        elif choice == "4":
            search_recipients()
        elif choice == "5":
            view_request_history()
        elif choice == "6":
            break
        else:
            print("Invalid choice. Please try again.")

def register_recipient():
    print("\nRegister New Recipient")
    name = input("Full Name: ")
    phone = input("Phone: ")
    email = input("Email: ")
    hospital = input("Hospital Name: ")
    doctor = input("Doctor Name: ")
    
    try:
        mycursor.execute("""
        INSERT INTO recipient (name, phone_no, email, hospital_name, doctor_name)
        VALUES (%s, %s, %s, %s, %s)
        """, (name, phone, email, hospital, doctor))
        recipient_id = mycursor.lastrowid
        mydb.commit()
        print(f"Recipient registered successfully. Recipient ID: {recipient_id}")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def update_recipient():
    recipient_id = input("Enter Recipient ID to update: ")
    
    try:
        mycursor.execute("SELECT * FROM recipient WHERE recipient_id = %s", (recipient_id,))
        recipient = mycursor.fetchone()
        
        if not recipient:
            print("Recipient not found")
            return
            
        print("\nCurrent Information:")
        print(f"1. Name: {recipient[1]}")
        print(f"2. Phone: {recipient[2]}")
        print(f"3. Email: {recipient[3]}")
        print(f"4. Hospital: {recipient[4]}")
        print(f"5. Doctor: {recipient[5]}")
        
        field = input("\nEnter field number to update (1-5): ")
        new_value = input("Enter new value: ")
        
        fields = {
            '1': 'name',
            '2': 'phone_no',
            '3': 'email',
            '4': 'hospital_name',
            '5': 'doctor_name'
        }
        
        if field in fields:
            query = f"UPDATE recipient SET {fields[field]} = %s WHERE recipient_id = %s"
            mycursor.execute(query, (new_value, recipient_id))
            mydb.commit()
            print("Recipient information updated successfully")
        else:
            print("Invalid field selection")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def view_all_recipients():
    try:
        mycursor.execute("""
        SELECT recipient_id, name, phone_no, email, hospital_name, doctor_name
        FROM recipient
        ORDER BY name
        """)
        recipients = mycursor.fetchall()
        
        print("\n{:<5} {:<25} {:<15} {:<25} {:<20} {:<20}".format(
            "ID", "Name", "Phone", "Email", "Hospital", "Doctor"))
        print("-"*110)
        
        for recipient in recipients:
            print("{:<5} {:<25} {:<15} {:<25} {:<20} {:<20}".format(
                recipient[0], recipient[1], recipient[2], 
                recipient[3], recipient[4], recipient[5]))
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def search_recipients():
    print("\nSearch Recipients By:")
    print("1. Name")
    print("2. Phone Number")
    print("3. Hospital")
    
    choice = input("Enter search option: ")
    
    try:
        if choice == "1":
            name = input("Enter Name (partial match): ")
            mycursor.execute("""
            SELECT recipient_id, name, phone_no, email, hospital_name
            FROM recipient
            WHERE name LIKE %s
            """, (f"%{name}%",))
        elif choice == "2":
            phone = input("Enter Phone Number: ")
            mycursor.execute("""
            SELECT recipient_id, name, phone_no, email, hospital_name
            FROM recipient
            WHERE phone_no LIKE %s
            """, (f"%{phone}%",))
        elif choice == "3":
            hospital = input("Enter Hospital Name: ")
            mycursor.execute("""
            SELECT recipient_id, name, phone_no, email, hospital_name
            FROM recipient
            WHERE hospital_name LIKE %s
            """, (f"%{hospital}%",))
        else:
            print("Invalid choice")
            return
            
        results = mycursor.fetchall()
        
        if not results:
            print("No recipients found")
            return
            
        print("\nSearch Results:")
        print("{:<5} {:<25} {:<15} {:<25} {:<20}".format(
            "ID", "Name", "Phone", "Email", "Hospital"))
        print("-"*90)
        
        for row in results:
            print("{:<5} {:<25} {:<15} {:<25} {:<20}".format(
                row[0], row[1], row[2], row[3], row[4]))
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def view_request_history():
    recipient_id = input("Enter Recipient ID: ")
    
    try:
        mycursor.execute("""
        SELECT r.request_id, r.blood_group, r.quantity_ml, r.request_date, 
               r.required_date, r.status, o.name
        FROM blood_request r
        JOIN organisation o ON r.oid = o.oid
        WHERE r.recipient_id = %s
        ORDER BY r.request_date DESC
        """, (recipient_id,))
        
        requests = mycursor.fetchall()
        
        if not requests:
            print("No request history found")
            return
            
        print("\nRequest History:")
        print("{:<5} {:<5} {:<8} {:<20} {:<20} {:<10} {:<20}".format(
            "ID", "BG", "Qty(ml)", "Request Date", "Required Date", "Status", "Organization"))
        print("-"*90)
        
        for req in requests:
            print("{:<5} {:<5} {:<8} {:<20} {:<20} {:<10} {:<20}".format(
                req[0], req[1], req[2], str(req[3]), str(req[4]), req[5], req[6]))
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def blood_request():
    print("\nBlood Request Process")
    recipient_id = input("Enter Recipient ID: ")
    
    try:
        mycursor.execute("SELECT * FROM recipient WHERE recipient_id = %s", (recipient_id,))
        recipient = mycursor.fetchone()
        
        if not recipient:
            print("Recipient not found")
            return
        
        print(f"\nRecipient: {recipient[1]}")
        print(f"Hospital: {recipient[4]}")
        print(f"Doctor: {recipient[5]}")
        
        blood_group = input("Required Blood Group: ").upper()
        quantity = int(input("Quantity needed (ml): "))
        required_date = input("Required by (YYYY-MM-DD): ")
        priority = input("Priority (Emergency/High/Normal): ").capitalize()
        doctor_approval = input("Doctor approval name: ")
        notes = input("Additional notes: ")
        
        mycursor.execute("SELECT oid, name, city FROM organisation")
        orgs = mycursor.fetchall()
        print("\nAvailable Organizations:")
        for org in orgs:
            print(f"{org[0]}: {org[1]} ({org[2]})")
        
        org_id = int(input("Select Organization ID: "))
        
        # After collecting org_id and blood_group from user:
        try:
            org_id = int(org_id)
        except ValueError:
            print("Invalid Organization ID")
            return
        
        blood_group = blood_group.strip().upper()
        
        mycursor.execute("""
        SELECT SUM(quantity_ml) 
        FROM blood_stock 
        WHERE blood_group = %s AND status = 'Available' AND expiry_date > CURDATE() AND oid = %s
        """, (blood_group, org_id))
        
        result = mycursor.fetchone()
        available = result[0] if result and result[0] is not None else 0
        
        if available >= quantity:
            print(f"Note: {available}ml of {blood_group} blood is currently available")
        else:
            print(f"Warning: Only {available}ml of {blood_group} blood available (requested {quantity}ml)")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    

def inventory_management():
    while True:
        print("\nInventory Management")
        print("1. View Current Inventory")
        print("2. Search Inventory")
        print("3. Update Inventory Status")
        print("4. Expire Old Stock")
        print("5. Back to Main Menu")
        
        choice = input("Enter choice: ")
        
        if choice == "1":
            view_inventory()
        elif choice == "2":
            search_inventory()
        elif choice == "3":
            update_inventory_status()
        elif choice == "4":
            expire_old_stock()
        elif choice == "5":
            break
        else:
            print("Invalid choice. Please try again.")

def view_inventory():
    try:
        mycursor.execute("""
        SELECT s.stock_id, s.blood_group, s.quantity_ml, s.collection_date, 
               s.expiry_date, s.status, o.name
        FROM blood_stock s
        JOIN organisation o ON s.oid = o.oid
        ORDER BY s.expiry_date
        """)
        
        inventory = mycursor.fetchall()
        
        if not inventory:
            print("No inventory records found")
            return
            
        print("\nCurrent Inventory:")
        print("{:<5} {:<5} {:<8} {:<15} {:<15} {:<10} {:<20}".format(
            "ID", "BG", "Qty(ml)", "Collection", "Expiry", "Status", "Organization"))
        print("-"*90)
        
        for item in inventory:
            print("{:<5} {:<5} {:<8} {:<15} {:<15} {:<10} {:<20}".format(
                item[0], item[1], item[2], str(item[3]), str(item[4]), item[5], item[6]))
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def search_inventory():
    print("\nSearch Inventory By:")
    print("1. Blood Group")
    print("2. Organization")
    print("3. Status")
    print("4. Expiry Date Range")
    
    choice = input("Enter search option: ")
    
    try:
        if choice == "1":
            bg = input("Enter Blood Group: ").upper()
            mycursor.execute("""
            SELECT s.stock_id, s.blood_group, s.quantity_ml, s.expiry_date, s.status, o.name
            FROM blood_stock s
            JOIN organisation o ON s.oid = o.oid
            WHERE s.blood_group = %s
            ORDER BY s.expiry_date
            """, (bg,))
        elif choice == "2":
            org = input("Enter Organization Name: ")
            mycursor.execute("""
            SELECT s.stock_id, s.blood_group, s.quantity_ml, s.expiry_date, s.status, o.name
            FROM blood_stock s
            JOIN organisation o ON s.oid = o.oid
            WHERE o.name LIKE %s
            ORDER BY s.expiry_date
            """, (f"%{org}%",))
        elif choice == "3":
            status = input("Enter Status (Available/Reserved/Expired/Used): ").capitalize()
            mycursor.execute("""
            SELECT s.stock_id, s.blood_group, s.quantity_ml, s.expiry_date, s.status, o.name
            FROM blood_stock s
            JOIN organisation o ON s.oid = o.oid
            WHERE s.status = %s
            ORDER BY s.expiry_date
            """, (status,))
        elif choice == "4":
            start = input("Enter Start Date (YYYY-MM-DD): ")
            end = input("Enter End Date (YYYY-MM-DD): ")
            mycursor.execute("""
            SELECT s.stock_id, s.blood_group, s.quantity_ml, s.expiry_date, s.status, o.name
            FROM blood_stock s
            JOIN organisation o ON s.oid = o.oid
            WHERE s.expiry_date BETWEEN %s AND %s
            ORDER BY s.expiry_date
            """, (start, end))
        else:
            print("Invalid choice")
            return
            
        results = mycursor.fetchall()
        
        if not results:
            print("No inventory items found")
            return
            
        print("\nSearch Results:")
        print("{:<5} {:<5} {:<8} {:<15} {:<10} {:<20}".format(
            "ID", "BG", "Qty(ml)", "Expiry", "Status", "Organization"))
        print("-"*70)
        
        for row in results:
            print("{:<5} {:<5} {:<8} {:<15} {:<10} {:<20}".format(
                row[0], row[1], row[2], str(row[3]), row[4], row[5]))
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def update_inventory_status():
    print("\nUpdate Blood Inventory Status")
    
    try:
        mycursor.execute("SELECT stock_id, blood_group, status FROM blood_stock")
        inventory = mycursor.fetchall()
        
        if not inventory:
            print("No inventory items found")
            return
            
        print("\nCurrent Blood Inventory:")
        for item in inventory:
            print(f"ID: {item[0]}, Blood Group: {item[1]}, Status: {item[2]}")
        
        stock_id = input("\nEnter Stock ID to update: ")
        
        mycursor.execute("SELECT stock_id, blood_group, status FROM blood_stock WHERE stock_id = %s", (stock_id,))
        stock_item = mycursor.fetchone()
        
        if not stock_item:
            print("Error: No inventory item found with that ID")
            return
        
        print(f"\nSelected Item:")
        print(f"Blood Group: {stock_item[1]}")
        print(f"Current Status: {stock_item[2]}")
        
        valid_statuses = ['Available', 'Reserved', 'Expired', 'Used']
        while True:
            new_status = input("Enter new status (Available/Reserved/Expired/Used): ").strip().capitalize()
            if new_status in valid_statuses:
                break
            print("Invalid status. Please choose from: Available, Reserved, Expired, Used")
        
        confirm = input(f"Confirm change status from '{stock_item[2]}' to '{new_status}'? (y/n): ").lower()
        if confirm != 'y':
            print("Update cancelled")
            return
        
        try:
            mycursor.execute("UPDATE blood_stock SET status = %s WHERE stock_id = %s", (new_status, stock_id))
            
            if not mydb.autocommit:
                mydb.commit()
                print("Inventory status updated successfully")
            else:
                print("Update completed (autocommit enabled)")
                
            mycursor.execute("SELECT status FROM blood_stock WHERE stock_id = %s", (stock_id,))
            updated_status = mycursor.fetchone()[0]
            print(f"Verification: Current status is now {updated_status}")
            
        except mysql.connector.Error as err:
            print(f"Failed to update: {err}")
            mydb.rollback()
            
    except Exception as e:
        print(f"Error: {e}")
        if 'mydb' in locals():
            mydb.rollback()

def expire_old_stock():
    confirm = input("Are you sure you want to mark all expired stock as expired? (y/n): ")
    
    if confirm.lower() != 'y':
        return
        
    try:
        mycursor.execute("""
        UPDATE blood_stock
        SET status = 'Expired'
        WHERE expiry_date <= CURDATE() AND status != 'Expired'
        """)
        
        count = mycursor.rowcount
        mydb.commit()
        print(f"Marked {count} expired stock items")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def reports_analytics():
    while True:
        print("\nReports & Analytics")
        print("1. Donation Summary")
        print("2. Blood Utilization Report")
        print("3. Donor Demographics")
        print("4. Organization Activity")
        print("5. Back to Main Menu")
        
        choice = input("Enter choice: ")
        
        if choice == "1":
            donation_summary()
        elif choice == "2":
            blood_utilization()
        elif choice == "3":
            donor_demographics()
        elif choice == "4":
            organization_activity()
        elif choice == "5":
            break
        else:
            print("Invalid choice. Please try again.")

def donation_summary():
    print("\nDonation Summary Report")
    print("1. By Date Range")
    print("2. By Organization")
    print("3. By Blood Group")
    
    choice = input("Enter report type: ")
    
    try:
        if choice == "1":
            start = input("Enter Start Date (YYYY-MM-DD): ")
            end = input("Enter End Date (YYYY-MM-DD): ")
            mycursor.execute("""
            SELECT DATE(donation_date) AS date, COUNT(*) AS donations, 
                   SUM(quantity_ml) AS total_ml, o.name
            FROM donation d
            JOIN organisation o ON d.oid = o.oid
            WHERE DATE(donation_date) BETWEEN %s AND %s
            GROUP BY DATE(donation_date), o.name
            ORDER BY date
            """, (start, end))
        elif choice == "2":
            mycursor.execute("""
            SELECT o.name, COUNT(*) AS donations, 
                   SUM(d.quantity_ml) AS total_ml, 
                   AVG(d.hemoglobin_level) AS avg_hemoglobin
            FROM donation d
            JOIN organisation o ON d.oid = o.oid
            GROUP BY o.name
            ORDER BY donations DESC
            """)
        elif choice == "3":
            mycursor.execute("""
            SELECT blood_group, COUNT(*) AS donations, 
                   SUM(quantity_ml) AS total_ml,
                   AVG(hemoglobin_level) AS avg_hemoglobin
            FROM donation
            GROUP BY blood_group
            ORDER BY blood_group
            """)
        else:
            print("Invalid choice")
            return
            
        results = mycursor.fetchall()
        
        if not results:
            print("No data found")
            return
            
        if choice == "1":
            print("\n{:<15} {:<10} {:<10} {:<20}".format(
                "Date", "Donations", "Total(ml)", "Organization"))
            print("-"*60)
            for row in results:
                print("{:<15} {:<10} {:<10} {:<20}".format(
                    str(row[0]), row[1], row[2], row[3]))
        elif choice == "2":
            print("\n{:<20} {:<10} {:<10} {:<10}".format(
                "Organization", "Donations", "Total(ml)", "Avg Hb"))
            print("-"*50)
            for row in results:
                print("{:<20} {:<10} {:<10} {:<10.1f}".format(
                    row[0], row[1], row[2], row[3]))
        elif choice == "3":
            print("\n{:<5} {:<10} {:<10} {:<10}".format(
                "BG", "Donations", "Total(ml)", "Avg Hb"))
            print("-"*35)
            for row in results:
                print("{:<5} {:<10} {:<10} {:<10.1f}".format(
                    row[0], row[1], row[2], row[3]))
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    

def blood_utilization():
    print("\nBlood Utilization Report")
    print("1. By Date Range")
    print("2. By Organization")
    print("3. By Blood Group")
    
    choice = input("Enter report type: ")
    
    try:
        if choice == "1":
            start = input("Enter Start Date (YYYY-MM-DD): ")
            end = input("Enter End Date (YYYY-MM-DD): ")
            mycursor.execute("""
            SELECT s.blood_group, 
                   SUM(CASE WHEN s.status = 'Used' THEN s.quantity_ml ELSE 0 END) AS used,
                   SUM(CASE WHEN s.status = 'Expired' THEN s.quantity_ml ELSE 0 END) AS expired,
                   SUM(CASE WHEN s.status = 'Available' THEN s.quantity_ml ELSE 0 END) AS available,
                   o.name
            FROM blood_stock s
            JOIN organisation o ON s.oid = o.oid
            WHERE s.collection_date BETWEEN %s AND %s
            GROUP BY s.blood_group, o.name
            ORDER BY s.blood_group
            """, (start, end))
        elif choice == "2":
            mycursor.execute("""
            SELECT o.name, s.blood_group, 
                   SUM(CASE WHEN s.status = 'Used' THEN s.quantity_ml ELSE 0 END) AS used,
                   SUM(CASE WHEN s.status = 'Expired' THEN s.quantity_ml ELSE 0 END) AS expired,
                   SUM(s.quantity_ml) AS total_collected
            FROM blood_stock s
            JOIN organisation o ON s.oid = o.oid
            GROUP BY o.name, s.blood_group
            ORDER BY o.name, s.blood_group
            """)
        elif choice == "3":
            mycursor.execute("""
            SELECT s.blood_group, 
                   SUM(CASE WHEN s.status = 'Used' THEN s.quantity_ml ELSE 0 END) AS used,
                   SUM(CASE WHEN s.status = 'Expired' THEN s.quantity_ml ELSE 0 END) AS expired,
                   SUM(s.quantity_ml) AS total_collected,
                   ROUND(SUM(CASE WHEN s.status = 'Used' THEN s.quantity_ml ELSE 0 END) / 
                         SUM(s.quantity_ml) * 100, 1) AS utilization_rate
            FROM blood_stock s
            GROUP BY s.blood_group
            ORDER BY s.blood_group
            """)
        else:
            print("Invalid choice")
            return
            
        results = mycursor.fetchall()
        
        if not results:
            print("No data found")
            return
            
        if choice == "1":
            print("\n{:<5} {:<10} {:<10} {:<10} {:<20}".format(
                "BG", "Used(ml)", "Expired(ml)", "Avail(ml)", "Organization"))
            print("-"*60)
            for row in results:
                print("{:<5} {:<10} {:<10} {:<10} {:<20}".format(
                    row[0], row[1], row[2], row[3], row[4]))
        elif choice == "2":
            print("\n{:<20} {:<5} {:<10} {:<10} {:<15}".format(
                "Organization", "BG", "Used(ml)", "Expired(ml)", "Total(ml)"))
            print("-"*60)
            for row in results:
                print("{:<20} {:<5} {:<10} {:<10} {:<15}".format(
                    row[0], row[1], row[2], row[3], row[4]))
        elif choice == "3":
            print("\n{:<5} {:<10} {:<10} {:<15} {:<10}%".format(
                "BG", "Used(ml)", "Expired(ml)", "Total(ml)", "Utilization"))
            print("-"*50)
            for row in results:
                print("{:<5} {:<10} {:<10} {:<15} {:<10}".format(
                    row[0], row[1], row[2], row[3], row[4]))
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def donor_demographics():
    print("\nDonor Demographics Report")
    print("1. By Age Group")
    print("2. By Blood Group")
    print("3. By Gender")
    print("4. Back to Reports")
    
    choice = input("Enter report type: ")
    
    try:
        if choice == "1":
            mycursor.execute("""
            SELECT 
                FLOOR(DATEDIFF(CURDATE(), dob)/365/10)*10 AS age_group,
                COUNT(*) AS donors,
                blood_group
            FROM donor
            GROUP BY age_group, blood_group
            ORDER BY age_group, blood_group
            """)
            
            results = mycursor.fetchall()
            print("\n{:<10} {:<5} {:<10}".format("Age Group", "BG", "Donors"))
            print("-"*30)
            for row in results:
                print("{:<10} {:<5} {:<10}".format(
                    f"{row[0]}-{row[0]+9}", row[2], row[1]))
                
        elif choice == "2":
            mycursor.execute("""
            SELECT 
                blood_group,
                COUNT(*) AS donors,
                AVG(weight) AS avg_weight,
                AVG(DATEDIFF(CURDATE(), last_donation_date)) AS avg_days_since_last
            FROM donor
            WHERE last_donation_date IS NOT NULL
            GROUP BY blood_group
            ORDER BY blood_group
            """)
            
            results = mycursor.fetchall()
            print("\n{:<5} {:<10} {:<15} {:<20}".format(
                "BG", "Donors", "Avg Weight", "Avg Days Since Last"))
            print("-"*50)
            for row in results:
                print("{:<5} {:<10} {:<15.1f} {:<20.1f}".format(
                    row[0], row[1], row[2], row[3]))
                
        elif choice == "3":
            mycursor.execute("""
            SELECT 
                gender,
                COUNT(*) AS donors,
                blood_group,
                AVG(DATEDIFF(CURDATE(), dob)/365) AS avg_age
            FROM donor
            GROUP BY gender, blood_group
            ORDER BY gender, blood_group
            """)
            
            results = mycursor.fetchall()
            print("\n{:<10} {:<5} {:<10} {:<10}".format(
                "Gender", "BG", "Donors", "Avg Age"))
            print("-"*40)
            for row in results:
                print("{:<10} {:<5} {:<10} {:<10.1f}".format(
                    row[0], row[2], row[1], row[3]))
                    
        elif choice == "4":
            return
        else:
            print("Invalid choice")
            
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def organization_activity():
    print("\nOrganization Activity Report")
    print("1. Donation Summary")
    print("2. Camp Summary")
    print("3. Inventory Status")
    
    choice = input("Enter report type: ")
    
    try:
        if choice == "1":
            mycursor.execute("""
            SELECT 
                o.name,
                COUNT(d.donation_id) AS donations,
                SUM(d.quantity_ml) AS total_ml,
                COUNT(DISTINCT d.donor_id) AS unique_donors
            FROM organisation o
            LEFT JOIN donation d ON o.oid = d.oid
            GROUP BY o.name
            ORDER BY donations DESC
            """)
            
            results = mycursor.fetchall()
            print("\n{:<30} {:<10} {:<10} {:<15}".format(
                "Organization", "Donations", "Total(ml)", "Unique Donors"))
            print("-"*65)
            for row in results:
                print("{:<30} {:<10} {:<10} {:<15}".format(
                    row[0] or "N/A", 
                    row[1] or 0, 
                    row[2] or 0, 
                    row[3] or 0))
                    
        elif choice == "2":
            mycursor.execute("""
            SELECT 
                o.name,
                COUNT(c.camp_id) AS camps,
                SUM(c.expected_donors) AS expected,
                SUM(c.actual_donors) AS actual,
                CASE 
                    WHEN SUM(c.expected_donors) = 0 THEN NULL
                    ELSE ROUND(SUM(c.actual_donors)/SUM(c.expected_donors)*100, 1)
                END AS fulfillment_rate
            FROM organisation o
            LEFT JOIN blood_camp c ON o.oid = c.oid
            GROUP BY o.name
            ORDER BY camps DESC
            """)
            
            results = mycursor.fetchall()
            print("\n{:<30} {:<10} {:<10} {:<10} {:<15}".format(
                "Organization", "Camps", "Expected", "Actual", "Fulfillment %"))
            print("-"*75)
            for row in results:
                fulfillment = f"{row[4]:.1f}%" if row[4] is not None else "N/A"
                print("{:<30} {:<10} {:<10} {:<10} {:<15}".format(
                    row[0] or "N/A", 
                    row[1] or 0, 
                    row[2] or 0, 
                    row[3] or 0, 
                    fulfillment))
                    
        elif choice == "3":
            mycursor.execute("""
            SELECT 
                o.name,
                s.blood_group,
                SUM(CASE WHEN s.status = 'Available' THEN s.quantity_ml ELSE 0 END) AS available,
                SUM(CASE WHEN s.status = 'Reserved' THEN s.quantity_ml ELSE 0 END) AS reserved,
                SUM(CASE WHEN s.status = 'Expired' THEN s.quantity_ml ELSE 0 END) AS expired,
                SUM(CASE WHEN s.status = 'Used' THEN s.quantity_ml ELSE 0 END) AS used
            FROM organisation o
            LEFT JOIN blood_stock s ON o.oid = s.oid
            GROUP BY o.name, s.blood_group
            ORDER BY o.name, s.blood_group
            """)
            
            results = mycursor.fetchall()
            print("\n{:<30} {:<5} {:<10} {:<10} {:<10} {:<10}".format(
                "Organization", "BG", "Available", "Reserved", "Expired", "Used"))
            print("-"*75)
            for row in results:
                print("{:<30} {:<5} {:<10} {:<10} {:<10} {:<10}".format(
                    row[0] or "N/A", 
                    row[1] or "N/A", 
                    row[2] or 0, 
                    row[3] or 0, 
                    row[4] or 0, 
                    row[5] or 0))
                    
        else:
            print("Invalid choice")
            
    except mysql.connector.Error as err:
        print(f"Database Error: {err}")
    except Exception as e:
        print(f"Error: {e}")

def system_config():
    while True:
        print("\nSystem Configuration")
        print("1. Add Organization")
        print("2. Manage Users")
        print("3. Back to Main Menu")
        
        choice = input("Enter choice (1-3): ")
        
        if choice == "1":
            add_organization()
        elif choice == "2":
            manage_users()
        elif choice == "3":
            break
        else:
            print("Invalid choice. Please try again.")

def add_organization():
    print("\nAdd New Organization")
    name = input("Organization Name: ")
    contact = input("Contact Number: ")
    email = input("Email: ")
    license_no = input("License Number: ")
    state = input("State: ")
    city = input("City: ")
    address = input("Address: ")
    established = input("Established Date (YYYY-MM-DD): ")
    
    try:
        mycursor.execute("""
        INSERT INTO organisation (name, contact_no, email, license_no, 
                                state, city, address, established_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (name, contact, email, license_no, state, city, address, established))
        mydb.commit()
        print("Organization added successfully")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def manage_users():
    while True:
        print("\nUser Management")
        print("1. List All Users")
        print("2. Add New User")
        print("3. Reset Password")
        print("4. Deactivate User")
        print("5. Back to System Config")
        
        choice = input("Enter choice: ")
        
        try:
            if choice == "1":
                mycursor.execute("""
                SELECT user_id, username, full_name, role, email, 
                       last_login, is_active
                FROM system_users
                ORDER BY role, username
                """)
                
                users = mycursor.fetchall()
                print("\n{:<5} {:<15} {:<20} {:<10} {:<25} {:<20} {:<8}".format(
                    "ID", "Username", "Full Name", "Role", "Email", "Last Login", "Active"))
                print("-"*100)
                for user in users:
                    print("{:<5} {:<15} {:<20} {:<10} {:<25} {:<20} {:<8}".format(
                        user[0], user[1], user[2], user[3], user[4], 
                        str(user[5]) if user[5] else "Never", "Yes" if user[6] else "No"))
                        
            elif choice == "2":
                print("\nAdd New User")
                username = input("Username: ")
                full_name = input("Full Name: ")
                password = input("Password: ")
                role = input("Role (Admin/Staff/Volunteer): ").capitalize()
                email = input("Email: ")
                
                mycursor.execute("""
                INSERT INTO system_users (username, password, full_name, role, email)
                VALUES (%s, %s, %s, %s, %s)
                """, (username, password, full_name, role, email))
                user_id = mycursor.lastrowid
                mydb.commit()
                print(f"User added successfully. User ID: {user_id}")
                
            elif choice == "3":
                username = input("Enter username to reset password: ")
                new_pass = input("Enter new password: ")
                
                mycursor.execute("""
                UPDATE system_users
                SET password = %s
                WHERE username = %s
                """, (new_pass, username))
                
                if mycursor.rowcount == 0:
                    print("User not found")
                else:
                    mydb.commit()
                    print("Password reset successfully")
                    
            elif choice == "4":
                username = input("Enter username to deactivate: ")
                
                mycursor.execute("""
                UPDATE system_users
                SET is_active = NOT is_active
                WHERE username = %s
                """, (username,))
                
                if mycursor.rowcount == 0:
                    print("User not found")
                else:
                    mydb.commit()
                    print("User status toggled successfully")
                    
            elif choice == "5":
                break
            else:
                print("Invalid choice")
                
        except mysql.connector.Error as err:
            print(f"Error: {err}")

if __name__ == "__main__":
    create_database()
    create_tables()
    insert_sample_data()
    create_procedures()
    create_triggers()
    main_menu()
    mydb.close()
                      

Connection Successful
Database 'blood_bank' created successfully
8 tables created successfully
Sample data inserted successfully (15 records in each table)
3 procedures created successfully
3 triggers created successfully

Blood Bank Management System
1. Donor Management
2. Recipient Management
3. Blood Donation
4. Blood Request
5. Inventory Management
6. Reports & Analytics
7. System Configuration
8. Exit
Enter choice (1-8): 1

Donor Management
1. Register New Donor
2. Update Donor Information
3. View All Donors
4. Search Donors
5. View Donation History
6. Back to Main Menu
Enter choice: 3

ID    Name                      Phone           Email                     BG    Last Donation   Active
----------------------------------------------------------------------------------------------------
10    Amanda Martinez           0123456789      amanda.m@email.com        B-    Never           Yes   
14    Ashley Clark              4455667788      ashley.c@email.com        B+    Never         